### Generates the Cohort using the criteria and defines the treatment groups

#### This is a copy of Elizabeth's notebook (ehealey/omop-learn/Manuscript_ready_files/Manuscript_Cohort_Generation) except I changed the following things

1)
"drug.drug_exposure_start_date - uti.condition_start_date < 8 AND"
TO
"drug.drug_exposure_start_date - uti.condition_start_date < 7 AND"

2) changed the antibiotic conditions (where the old were the comments) :
The code is under the line (for idx, row in anti_new.iterrows)

3)
all the dataframes created found by looking for "db_execute and to_sql" have an a "_ncj" tag appended to them


4) changed reload_antibiotic "= 0" to "= 1"

5) added this conditiional change: "if 'level_0' in cohort_treatments_new.columns:"
    cohort_treatments_new =cohort_treatments_new.drop(columns=['level_0'])
cohort_treatments_new.to_sql("uti_candidate_events_with_treatment_ncj",con=db.engine, if_exists="replace", schema="cdm_6871_21")

In [1]:
import sys

sys.path.append( '..' )

In [2]:
import psycopg2
import pandas as pd
import numpy
import sparse
import config
import sys
import time
import importlib
#import sparse
import datetime
import pandas as pd
import numpy as np
import scipy.sparse
import matplotlib.pyplot as plt

import Utils.dbutils as dbutils
import Utils.data_utils as data_utils
import Generators.CohortGenerator as CohortGenerator
import Generators.FeatureGenerator as FeatureGenerator
import config
local_imports = (
    dbutils,
    data_utils,
    CohortGenerator,
    FeatureGenerator,
    config
)
for i in local_imports:
    i = importlib.reload(i)

In [5]:
# database connection
username = config.PG_USERNAME
password = config.PG_PASSWORD
database_name = config.DB_NAME

#config_path = 'postgresql://{username}:{password}@{database_name}'.format(
config_path = 'postgresql://{database_name}'.format(database_name = database_name)

connect_args = {"host": '/var/run/postgresql/'} # connect_args to pass to sqlalchemy create_engine function

# schemas 
# schema_name = 'eol_test_ncjones' # all created tables will be created using this schema
cdm_schema_name = config.OMOP_CDM_SCHEMA # the name of the schema housing your OMOP CDM tables
print(f"cdm schema: {cdm_schema_name}")

    #username = username,
    #password = password,
   # database_name = database_name
#)

# schemas 
schema_name = 'eol_test_eheal' # all created tables will be created using this schema

# caching
reset_schema = False # if true, rebuild all data from scratch

# set up database, reset schemas as needed
db = dbutils.Database(config_path, schema_name, connect_args, cdm_schema_name)
# if reset_schema:
#     db.execute(
#         'drop schema if exists {} cascade'.format(schema_name)
#     )
# db.execute(
#     'create schema if not exists {}'.format(schema_name)
# )

username='cdm_6871_21'
password='MLHC2021'
database_name = 'omop_v6'
conn_string = "dbname="+database_name + " host=/var/run/postgresql"
conn=psycopg2.connect(conn_string)

cdm schema: cdm_6871_21


## UTI

In [6]:
# Get all female condition events

In [7]:
%%time
sql = """
    select 
        a.condition_concept_id as concept_id,
        a.condition_start_date as condition_start_date,
        a.condition_occurrence_id as condition_occurrence_id,
        p.person_id as person_id
    from
        {omop_schema}.condition_occurrence a
    inner join
        {omop_schema}.person p
    on 
        a.person_id = p.person_id
    where 
        p.gender_source_value = 'F'

""".format(
    omop_schema=config.OMOP_CDM_SCHEMA,
    training_end_date='2020-01-01',
)
condition_df = db.query(sql)
condition_df.set_index(["condition_occurrence_id"])
display(condition_df.head())

,concept_id,condition_start_date,condition_occurrence_id,person_id
0,443744,2015-10-09,170574397,13
1,437833,2014-12-09,110201808,13
2,80809,2015-12-10,183780400,13
3,254761,2014-04-28,69378391,13
4,79106,2014-06-23,78917016,13


CPU times: user 6min 56s, sys: 2min 5s, total: 9min 2s
Wall time: 16min 29s


In [8]:
'''%%time
sql = """
    select 
        a.condition_source_value as concept_code,
        a.condition_concept_id as concept_id,
        a.condition_start_date as condition_start_date,
        a.condition_occurrence_id as condition_occurrence_id,
        p.person_id as person_id
    from
        {omop_schema}.condition_occurrence a
    inner join
        {omop_schema}.person p
    on 
        a.person_id = p.person_id
    where 
        p.gender_source_value = 'F'

""".format(
    omop_schema=config.OMOP_CDM_SCHEMA,
    training_end_date='2020-01-01',
)
condition_df = db.query(sql)
condition_df.set_index(["condition_occurrence_id"])
display(condition_df.head())'''

'%%time\nsql = """\n    select \n        a.condition_source_value as concept_code,\n        a.condition_concept_id as concept_id,\n        a.condition_start_date as condition_start_date,\n        a.condition_occurrence_id as condition_occurrence_id,\n        p.person_id as person_id\n    from\n        {omop_schema}.condition_occurrence a\n    inner join\n        {omop_schema}.person p\n    on \n        a.person_id = p.person_id\n    where \n        p.gender_source_value = \'F\'\n\n""".format(\n    omop_schema=config.OMOP_CDM_SCHEMA,\n    training_end_date=\'2020-01-01\',\n)\ncondition_df = db.query(sql)\ncondition_df.set_index(["condition_occurrence_id"])\ndisplay(condition_df.head())'

In [9]:
len(condition_df)

162247860

In [10]:
## Get UTI Codes

In [11]:
%%time
uti_concept_ids = [194081, 
                   195588,
                   37018854,
                   37016338,
                   81902]

uti_string = ", ".join([f"'{x}'" for x in uti_concept_ids])
print(uti_string)

'194081', '195588', '37018854', '37016338', '81902'
CPU times: user 282 µs, sys: 85 µs, total: 367 µs
Wall time: 292 µs


In [19]:
#       81902 | Urinary tract infectious disease             | Condition | SNOMED        | Clinical Finding | S                | 68566005        | 1970-01-01       | 2099-12-31     | n
#      194081 | Acute cystitis                                | Condition | SNOMED        | Clinical Finding | S                | 68226007        | 1970-01-01       | 2099-12-31     | n
#      195588 | Cystitis                                      | Condition | SNOMED        | Clinical Finding | S                | 38822007        | 1970-01-01       | 2099-12-31     | n
#      195862 | Urethritis (REMOVED)                          | Condition | SNOMED        | Clinical Finding | S                | 31822004        | 1970-01-01       | 2099-12-31     | n
#      201353 | Irradiation cystitis (REMOVED)                | Condition | SNOMED        | Clinical Finding | S                | 11251000        | 1970-01-01       | 2099-12-31     | n
#      201792 | Nongonococcal urethritis (REMOVED)            | Condition | SNOMED        | Clinical Finding | S                | 84619001        | 1970-01-01       | 2099-12-31     | n
#    37016338 | Hematuria co-occurrent and due to cystitis       | Condition | SNOMED        | Clinical Finding | S                | 367171000119100 | 2016-01-31       | 2099-12-31     | n
#    37018854 | Hematuria co-occurrent and due to acute cystitis | Condition | SNOMED        | Clinical Finding | S                | 366941000119107 | 2016-01-31       | 2099-12-31     | n

In [12]:
%%time
# Filter by UTI concept ids in python to reduce query time
utis = condition_df[condition_df.concept_id.isin(uti_concept_ids)]
print(len(utis))
utis.head()

1462322
CPU times: user 1.22 s, sys: 807 ms, total: 2.03 s
Wall time: 2.03 s


,concept_id,condition_start_date,condition_occurrence_id,person_id
194,81902,2015-01-20,113407091,24
239,81902,2018-06-30,400205390,29
288,81902,2018-06-30,400169196,29
365,81902,2020-06-15,543562412,35
481,81902,2020-06-15,543729870,35


# get treatment information for all UTIs


In [13]:
# remove duplicates of dates 
utis_all = utis.drop_duplicates(subset= ['concept_id','person_id','condition_start_date'],keep='first', inplace=False,)


In [22]:
utis_all.to_sql("candidate_events_all_ncj",con=db.engine, if_exists="replace", schema="cdm_6871_21")



In [5]:
%%time
reload_antibiotic=1
username='cdm_6871_21'
password='MLHC2021'
database_name = 'omop_v6'
conn_string = "dbname="+database_name + " host=/var/run/postgresql"
conn=psycopg2.connect(conn_string)

if reload_antibiotic == 1:
    all_drugs = pd.read_sql("""SELECT * FROM cdm_6871_21.concept
                                 WHERE domain_id = 'Drug'""", conn)

### Identify the antibiotic in each drug name 
    # First Line
    all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("nitrofurantoin"), 'antibiotic_name'] = "nitrofurantoin"
    all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("sulfamethoxazole"), 'antibiotic_name'] = "sulfamethoxazole"
#     all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("trimethoprim"), 'antibiotic_name'] = "trimethoprim"
    all_drugs.loc[(all_drugs["concept_name"].str.lower().str.contains("trimethoprim")) & \
                                 (all_drugs["concept_name"].str.lower().str.contains("sulfamethoxazole")), 'antibiotic_name'] = "trimethoprim-sulfamethoxazole"
    all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("fosfomycin"), 'antibiotic_name'] = "fosfomycin"

    # Second Line
#     all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("cephalexin"), 'antibiotic_name'] = "cephalexin"
#     all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("cephalexin"), 'antibiotic_name'] = "quinolone"


#     all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("ceftriaxone"), 'antibiotic_name'] = "ceftriaxone"
#     all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("cefuroxime"), 'antibiotic_name'] = "cefuroxime"
#     all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("cefdinir"), 'antibiotic_name'] = "cefdinir"
#     all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("cefazolin"), 'antibiotic_name'] = "cefazolin"
#     all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("cefepime"), 'antibiotic_name'] = "cefepime"
    all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("cefpodoxime"), 'antibiotic_name'] = "cefpodoxime"
#     all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("cefixime"), 'antibiotic_name'] = "cefixime"
    all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("cefadroxil"), 'antibiotic_name'] = "cefadroxil"
#     all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("amoxicillin"), 'antibiotic_name'] = "amoxicillin"
    all_drugs.loc[(all_drugs["concept_name"].str.lower().str.contains("amoxicillin")) & \
                                 (all_drugs["concept_name"].str.lower().str.contains("clavulan")), 'antibiotic_name'] = "amoxicillin-clavulanic acid"

    # Third Line

    all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("ofloxacin"), 'antibiotic_name'] = "ofloxacin"
    all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("levofloxacin"), 'antibiotic_name'] = "levofloxacin"
    all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("ciprofloxacin"), 'antibiotic_name'] = "ciprofloxacin"

    # Inappropriate

    all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("fluconazole"), 'antibiotic_name'] = "fluconazole"
    all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("doxycycline"), 'antibiotic_name'] = "doxycycline"
    all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("clindamycin"), 'antibiotic_name'] = "clindamycin"
    all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("linezolid"), 'antibiotic_name'] = "linezolid"
    all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("metronidazole"), 'antibiotic_name'] = "metronidazole"
    all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("azithromycin"), 'antibiotic_name'] = "azithromycin"
    
    # Alternative
    all_drugs.loc[all_drugs["concept_name"].str.lower().str.contains("beta-lactam"), 'antibiotic_name'] = 'beta-lactam'


    all_drugs = all_drugs.rename(columns={"concept_id": "drug_concept_id", "concept_name": "drug_name"})

    anti_new = all_drugs[['drug_concept_id','drug_name','antibiotic_name']]
    anti_new = anti_new[~anti_new.antibiotic_name.isna()]
    if 'level_0' in anti_new.columns:
        anti_new = anti_new.drop(columns=['level_0'])
    anti_new.to_sql("antibiotics_p1_ncj",con=db.engine, if_exists="replace", schema="cdm_6871_21")

if reload_antibiotic == 0:
    sql_command = "select * from cdm_6871_21.antibiotics_p1_ncj;"
    anti_new = pd.read_sql(sql_command,conn)


anti_new = anti_new.assign(nitrofuantoin=0)
anti_new = anti_new.assign(fosfomycin=0)
anti_new['trimethoprim-sulfamethoxazole'] =0
anti_new = anti_new.assign(second_line=0)
#anti_new = anti_new.assign(third_line=0)
anti_new = anti_new.assign(alternatives=0)
anti_new = anti_new.assign(inappropriate=0)
anti_new = anti_new.assign(antibiotic_type='')

###
count = 0
for idx, row in anti_new.iterrows():
    drug = row.antibiotic_name 
    if drug =='nitrofurantoin':
        anti_new.loc[idx, "nitrofurantoin"] = 1
        anti_new.loc[idx, "antibiotic_type"] ="nitrofurantoin"
    elif drug =='fosfomycin':
        anti_new.loc[idx, "fosfomycin"] = 1
        anti_new.loc[idx, "antibiotic_type"] ="fosfomycin"
#     elif drug =='trimethoprim-sulfamethoxazole' or drug =='trimethoprim' or drug =='sulfamethoxazole':
    elif drug =='trimethoprim-sulfamethoxazole':
        anti_new.loc[idx, 'trimethoprim-sulfamethoxazole'] = 1
        anti_new.loc[idx, "antibiotic_type"] ='trimethoprim-sulfamethoxazole'

       
    #     elif ("cef" in drug.lower()) or drug =='cephalexin' or ("amoxicillin" in drug.lower()):
    elif drug.lower() in ["amoxicillin-clavulanic acid","cefpodoxime","cefadroxil"]:
        anti_new.loc[idx, 'alternatives'] = 1
        anti_new.loc[idx, "antibiotic_type"] ='alternatives'
        
#     elif drug == "quinolone":
#         anti_new.loc[idx, 'second_line'] = 1
#         anti_new.loc[idx, "antibiotic_type"] ='second_line'

#     elif 'ofloxacin' in drug.lower() :
    elif drug.lower() in ["ciprofloxacin","ofloxacin","levofloxacin"]:
        anti_new.loc[idx, 'second_line'] = 1
        anti_new.loc[idx, "antibiotic_type"] ='second_line'
    elif drug== 'fluconazole' or drug== 'doxycycline'or drug== 'clindamycin' or drug== 'linezolid' or drug== 'metronidazole' or drug =='azithromycin' or 'injection' in drug:
        anti_new.loc[idx, 'inappropriate'] = 1
        anti_new.loc[idx, "antibiotic_type"] ='inappropriate'


    '''    elif 'beta-lactam' in drug.lower() :
        anti_new.loc[idx, 'alternatives'] = 1
        anti_new.loc[idx, "antibiotic_type"] ='alternatives'
    '''

    count = count+1
    if count%5000 ==0:
        print(round(count/len(anti_new),2))
        
# remove patients with fosfomycin 

anti_new = anti_new[anti_new['fosfomycin']==0]
anti_new.to_sql("antibiotics_v2_ncj",con=db.engine, if_exists="replace", schema="cdm_6871_21")

0.06
0.13
0.19
0.25
0.32
0.38
0.45
0.51
0.57
0.64
0.7
0.76
0.83
0.89
0.95
CPU times: user 2min 14s, sys: 152 ms, total: 2min 14s
Wall time: 2min 14s


In [41]:
%%time
# reload_antibiotic=0
username='cdm_6871_21'
password='MLHC2021'
database_name = 'omop_v6'
conn_string = "dbname="+database_name + " host=/var/run/postgresql"
conn=psycopg2.connect(conn_string)

CPU times: user 2.51 ms, sys: 0 ns, total: 2.51 ms
Wall time: 13.6 ms


In [47]:
%%time
cohort_treatments = pd.read_sql("""SELECT
                                uti.index,
                                uti.condition_occurrence_id, 
                                uti.person_id,
                                uti.concept_id,
                                uti.condition_start_date,
                                anti.*,
                                drug.drug_exposure_id,
                                drug.drug_exposure_start_date,
                                drug.drug_exposure_start_datetime
                           FROM cdm_6871_21.candidate_events_all_ncj uti

                           JOIN cdm_6871_21.drug_exposure drug ON 
                                       drug.person_id = uti.person_id AND
                                       drug.drug_exposure_start_date - uti.condition_start_date < 7 AND
                                       drug.drug_exposure_start_date - uti.condition_start_date > -1
                           JOIN cdm_6871_21.antibiotics_v2_ncj anti ON
                                       drug.drug_concept_id = anti.drug_concept_id
                           ORDER BY 
                                    uti.condition_occurrence_id
                        """, conn)

CPU times: user 3.18 s, sys: 145 ms, total: 3.32 s
Wall time: 1min 36s


In [124]:
#Remove empty antibiotic types
if '' in cohort_treatments.antibiotic_type.tolist():
    cohort_treatments = cohort_treatments.copy().loc[cohort_treatments.antibiotic_type != '']

In [48]:
%%time
# Load antibiotic use in the previous 7 days 
cohort_treatments_drugs_past_7_day = pd.read_sql("""SELECT
                                uti.index,
                                uti.condition_occurrence_id, 
                                uti.person_id,
                                uti.concept_id,
                                uti.condition_start_date,
                                anti.*,
                                drug.drug_exposure_id,
                                drug.drug_exposure_start_date,
                                drug.drug_exposure_start_datetime
                           FROM cdm_6871_21.candidate_events_all_ncj uti

                           JOIN cdm_6871_21.drug_exposure drug ON 
                                       drug.person_id = uti.person_id AND
                                       drug.drug_exposure_start_date - uti.condition_start_date > -8 AND
                                       drug.drug_exposure_start_date - uti.condition_start_date < 0
                           JOIN cdm_6871_21.antibiotics_v2_ncj anti ON
                                       drug.drug_concept_id = anti.drug_concept_id
                           ORDER BY 
                                    uti.condition_occurrence_id
                        """, conn)

CPU times: user 850 ms, sys: 43.6 ms, total: 894 ms
Wall time: 1min 32s


In [49]:
#UTI events with treatment
uti_unique = cohort_treatments
uti_unique = uti_unique.drop_duplicates(subset = ['condition_occurrence_id'], inplace = False, keep='first')
print('Unique UTI events with treatment')
print(len(uti_unique))


#UTI events with treatment but without treatment 7 days prior
condition_occurrence_id_to_exclude = cohort_treatments_drugs_past_7_day.condition_occurrence_id.unique()
cohort_treatments_new = cohort_treatments[~cohort_treatments.condition_occurrence_id.isin(condition_occurrence_id_to_exclude)]
uti_unique = cohort_treatments_new
uti_unique = uti_unique.drop_duplicates(subset = ['condition_occurrence_id'], inplace = False, keep='first')
print('Unique UTI events with treatment and without treatment 7 days prior')
print(len(uti_unique))



Unique UTI events with treatment
245368
Unique UTI events with treatment and without treatment 7 days prior
223456


In [54]:
if 'level_0' in cohort_treatments_new.columns:
    cohort_treatments_new =cohort_treatments_new.drop(columns=['level_0'])
cohort_treatments_new.to_sql("uti_candidate_events_with_treatment_ncj",con=db.engine, if_exists="replace", schema="cdm_6871_21")

In [55]:
%%time
# Get the full condition item table
candidate_indices = cohort_treatments_new.condition_occurrence_id
candidate_indices_str = ", ".join([f"'{x}'" for x in candidate_indices])

sql = """
    select
        c.*,
        p.*
    from
        {omop_schema}.condition_occurrence c
    inner join
        {omop_schema}.uti_candidate_events_with_treatment_ncj p
    on 
        c.condition_occurrence_id = p.condition_occurrence_id
    where 
        c.condition_occurrence_id in ({candidate_indices_str}) 
""".format(
    omop_schema=config.OMOP_CDM_SCHEMA,
    candidate_indices_str=candidate_indices_str,
)
condition_candidates = db.query(sql)


CPU times: user 6.59 s, sys: 196 ms, total: 6.78 s
Wall time: 30min 25s


In [56]:
if 'level_0' in list(condition_candidates.keys()):
    condition_candidates=condition_candidates.drop(columns=['level_0'])


condition_candidates.to_sql("uti_candidate_events_treatment_ncj",con=db.engine, if_exists="replace", schema="cdm_6871_21")
db.execute("GRANT SELECT ON TABLE cdm_6871_21.uti_candidate_events_treatment_ncj TO cdm_6871_21;")

Executed 1 SQLs


In [6]:
%%time
cohort_raw = pd.read_sql("""SELECT
                                uti.index,
                                uti.condition_occurrence_id, 
                                uti.person_id,
                                uti.condition_concept_id,
                                uti.condition_start_date,
                                uti.condition_start_datetime,
                                uti.condition_end_date,
                                uti.condition_end_datetime,
                                uti.visit_occurrence_id,
                                uti.visit_detail_id,
                                anti.drug_concept_id, 
                                anti.drug_name,
                                anti.antibiotic_name,
                                anti.antibiotic_type,
                                v.provider_id AS visit_provider_id,
                                drug.drug_exposure_id,
                                drug.drug_exposure_start_date,
                                drug.drug_exposure_start_datetime,
                                drug.provider_id AS provider_id,
                                prov.provider_name,
                                prov.npi
                           FROM cdm_6871_21.uti_candidate_events_treatment_ncj uti
                           JOIN cdm_6871_21.visit_detail v ON 
                                   v.visit_detail_id = uti.visit_detail_id AND
                                   v.visit_detail_concept_id IN (581477, 9202)
                           JOIN cdm_6871_21.drug_exposure drug ON 
                                       drug.person_id = uti.person_id AND
                                       drug.drug_exposure_start_date - uti.condition_start_date < 7 AND
                                       drug.drug_exposure_start_date - uti.condition_start_date > -1
                           JOIN cdm_6871_21.antibiotics_v2_ncj anti ON
                                       drug.drug_concept_id = anti.drug_concept_id
                           LEFT JOIN cdm_6871_21.provider prov
                                    ON prov.provider_id = drug.provider_id
                           ORDER BY 
                                    uti.condition_occurrence_id,
                                    prov.provider_name DESC NULLS LAST
                        """, conn)



CPU times: user 3.57 s, sys: 496 ms, total: 4.07 s
Wall time: 3h 1min 52s


In [7]:
print('Size of df:')
print(len(cohort_raw))

Size of df:
329547


In [ ]:
#here

In [8]:
print('Size of df:')
print(len(cohort_raw))


Size of df:
329547


In [9]:
uti_unique = cohort_raw
uti_unique = uti_unique.drop_duplicates(subset = ['condition_occurrence_id','antibiotic_name'], inplace = False, keep='first')

print('Number of UTI events with treatment + proper visit:')
print(uti_unique.condition_occurrence_id.nunique())

Number of UTI events with treatment + proper visit:
187666


In [10]:
utis_with_treatment = cohort_raw

### find non-recurrent UTIs
use utis_all

In [11]:
utis = utis_with_treatment.copy()

In [12]:
utis = utis.sort_values(by=["person_id", "condition_start_date"], ascending=True)
utis = utis.assign(no_previous_180_day_event=True)

utis = utis.assign(no_two_previous_365_day_event=True)
utis = utis.assign(post_UTI_codes=0)
utis = utis.assign(recurrent_uti=False)
utis = utis.assign(first_uti=False)

utis = utis.assign(previous_uti=False)
utis = utis.assign(previous_utis=0)
utis = utis.assign(days_since_previous_uti=np.nan)
utis = utis.assign(previous_uti_condition_occurence_id=0)
utis = utis.assign(previous_uti_recurrent=False)


utis.head()

,index,condition_occurrence_id,person_id,condition_concept_id,condition_start_date,condition_start_datetime,condition_end_date,condition_end_datetime,visit_occurrence_id,visit_detail_id,...,no_previous_180_day_event,no_two_previous_365_day_event,post_UTI_codes,recurrent_uti,first_uti,previous_uti,previous_utis,days_since_previous_uti,previous_uti_condition_occurence_id,previous_uti_recurrent
246247,213958,400205390,29,81902,2018-06-30,2018-06-30,None,None,912651,305167035,...,True,True,0,False,False,False,0,NaN,0,False
246248,3828990,400205390,29,81902,2018-06-30,2018-06-30,None,None,912651,305167035,...,True,True,0,False,False,False,0,NaN,0,False
246249,3828990,400205390,29,81902,2018-06-30,2018-06-30,None,None,912651,305167035,...,True,True,0,False,False,False,0,NaN,0,False
246250,213958,400205390,29,81902,2018-06-30,2018-06-30,None,None,912651,305167035,...,True,True,0,False,False,False,0,NaN,0,False
318089,285902,543518058,35,81902,2020-06-14,2020-06-14,None,None,912779,418770757,...,True,True,0,False,False,False,0,NaN,0,False


#### get rid of multiple drugs

In [13]:
uti_unique = utis.copy()
print('Number of UTI events + individual drugs')
print(len(uti_unique))

Number of UTI events + individual drugs
329547


In [14]:
# drop duplicate drug types
uti_unique = uti_unique.drop_duplicates(subset = ['condition_occurrence_id','antibiotic_name'], inplace = False, keep='first')



In [15]:
multi =pd.DataFrame(uti_unique['condition_occurrence_id'].value_counts()).reset_index()
condition_occurence_ids = list(multi[multi['condition_occurrence_id']>1]['index'])
print('IDs with multiple drugs:')
print(len(condition_occurence_ids))

IDs with multiple drugs:
22124


In [16]:
%%time
uti_unique = uti_unique.assign(multi = False)
count = 0
for ind, row in uti_unique.iterrows():
    if row.condition_occurrence_id in condition_occurence_ids:
        uti_unique.loc[ind,'multi'] = True
    count = count+1
    if count%5000==0:
        print(count/len(uti_unique))
        

0.023617212224269046
0.04723442444853809
0.07085163667280714
0.09446884889707619
0.11808606112134523
0.14170327334561428
0.16532048556988332
0.18893769779415237
0.21255491001842142
0.23617212224269046
0.2597893344669595
0.28340654669122856
0.3070237589154976
0.33064097113976665
0.3542581833640357
0.37787539558830474
0.4014926078125738
0.42510982003684283
0.4487270322611119
0.4723442444853809
0.49596145670965
0.519578668933919
0.5431958811581881
0.5668130933824571
0.5904303056067262
0.6140475178309952
0.6376647300552643
0.6612819422795333
0.6848991545038023
0.7085163667280714
0.7321335789523404
0.7557507911766095
0.7793680034008785
0.8029852156251476
0.8266024278494166
0.8502196400736857
0.8738368522979547
0.8974540645222238
0.9210712767464928
0.9446884889707619
0.9683057011950309
0.9919229134193
CPU times: user 1min 49s, sys: 473 ms, total: 1min 49s
Wall time: 1min 49s


In [17]:
# Drop multi drugs
uti_unique = uti_unique.drop_duplicates(subset = ['condition_occurrence_id'], inplace = False, keep='first')

utis = uti_unique.copy()

#### Get non_recurrent UTIs, and note previous UTI that had treatment

In [18]:
'''utis = utis.sort_values(by=["person_id", "condition_start_date"], ascending=True)
utis = utis.assign(no_previous_180_day_event=True)

utis = utis.assign(no_two_previous_365_day_event=True)
utis = utis.assign(post_UTI_codes=0)
utis = utis.assign(recurrent_uti=False)
utis = utis.assign(first_uti=False)

utis = utis.assign(previous_uti=False)
utis = utis.assign(previous_utis=0)
utis = utis.assign(days_since_previous_uti=np.nan)
utis = utis.assign(previous_uti_condition_occurence_id=0)
utis = utis.assign(previous_uti_recurrent=False)


utis.head()'''

'utis = utis.sort_values(by=["person_id", "condition_start_date"], ascending=True)\nutis = utis.assign(no_previous_180_day_event=True)\n\nutis = utis.assign(no_two_previous_365_day_event=True)\nutis = utis.assign(post_UTI_codes=0)\nutis = utis.assign(recurrent_uti=False)\nutis = utis.assign(first_uti=False)\n\nutis = utis.assign(previous_uti=False)\nutis = utis.assign(previous_utis=0)\nutis = utis.assign(days_since_previous_uti=np.nan)\nutis = utis.assign(previous_uti_condition_occurence_id=0)\nutis = utis.assign(previous_uti_recurrent=False)\n\n\nutis.head()'

In [19]:
print('UTI events, with multiple drugs denoted:')
print(len(utis))

UTI events, with multiple drugs denoted:
187666


In [20]:
%%time
import datetime
date_format = "%Y-%m-%d"
last_patient = None
last_date = None
d_old = None
c = 0
patient_count = 0
for index, row in utis.iterrows():
    p = row.person_id
    d = row.condition_start_date
    cond = row.condition_occurrence_id

    #days_since_last = (d - last_date).days
    if last_patient ==p:
        days_since_last = (d - last_date).days
        if days_since_last < 180:
            utis.loc[index, "no_previous_180_day_event"] = False
        if second_last_patient ==p:
            if days_since_last < 365 and (d - second_last_date).days < 365 and (last_date-second_last_date).days>14:
                utis.loc[index, "no_two_previous_365_day_event"] = False

        #utis.loc[index, "recurrent_uti"] = True
    # keep track of previous UTI 
    
        patient_count = patient_count+1
        utis.loc[index, "previous_uti"] = True
        utis.loc[index, "days_since_previous_uti"] = days_since_last
        utis.loc[index, "previous_utis"] = patient_count
        utis.loc[index, "previous_uti_condition_occurence_id"] = last_cond
        if ~((last_row["no_two_previous_365_day_event"]) * (last_row["no_previous_180_day_event"])):
            utis.loc[index, "previous_uti_recurrent"] = True
    else:
        patient_count = 0
    second_last_patient = last_patient       
    last_patient = p
    last_date = d
    second_last_date = d_old
    d_old = d
    last_cond = cond
    last_row = row
    c += 1
    if c%1000 == 0:
        print(c/len(utis))
print(len(utis))
utis.head(2)

0.0053286157322050874
0.010657231464410175
0.015985847196615263
0.02131446292882035
0.02664307866102544
0.031971694393230526
0.037300310125435616
0.0426289258576407
0.04795754158984579
0.05328615732205088
0.05861477305425596
0.06394338878646105
0.06927200451866614
0.07460062025087123
0.07992923598307632
0.0852578517152814
0.0905864674474865
0.09591508317969158
0.10124369891189666
0.10657231464410176
0.11190093037630684
0.11722954610851193
0.12255816184071702
0.1278867775729221
0.1332153933051272
0.13854400903733227
0.14387262476953738
0.14920124050174247
0.15452985623394755
0.15985847196615263
0.16518708769835772
0.1705157034305628
0.1758443191627679
0.181172934894973
0.18650155062717808
0.19183016635938316
0.19715878209158824
0.20248739782379332
0.20781601355599844
0.21314462928820352
0.2184732450204086
0.22380186075261368
0.22913047648481877
0.23445909221702385
0.23978770794922896
0.24511632368143405
0.2504449394136391
0.2557735551458442
0.2611021708780493
0.2664307866102544
0.271759

,index,condition_occurrence_id,person_id,condition_concept_id,condition_start_date,condition_start_datetime,condition_end_date,condition_end_datetime,visit_occurrence_id,visit_detail_id,...,no_two_previous_365_day_event,post_UTI_codes,recurrent_uti,first_uti,previous_uti,previous_utis,days_since_previous_uti,previous_uti_condition_occurence_id,previous_uti_recurrent,multi
246247,213958,400205390,29,81902,2018-06-30,2018-06-30,None,None,912651,305167035,...,True,0,False,False,False,0,NaN,0,False,True
318089,285902,543518058,35,81902,2020-06-14,2020-06-14,None,None,912779,418770757,...,True,0,False,False,False,0,NaN,0,False,False


In [21]:
utis['recurrent_uti'] = ~((utis["no_two_previous_365_day_event"]) * (utis["no_previous_180_day_event"]))

/home/ncjones/miniconda3/envs/omop/lib/python3.8/site-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(


In [22]:
non_recurrent_utis = utis[utis['recurrent_uti']==False ]
#non_recurrent_utis = utis[utis['first_uti']==True ]
print('Number of non-recurrent UTI events:')
print(len(non_recurrent_utis))
current_df = non_recurrent_utis

Number of non-recurrent UTI events:
140694


### Remove patients under 18 at time of UTI

In [23]:
%%time
username='cdm_6871_21'
password='MLHC2021'
database_name = 'omop_v6'
conn_string = "dbname="+database_name + " host=/var/run/postgresql"
conn=psycopg2.connect(conn_string)
print('Connected!')

sql_command = "select * from cdm_6871_21.person;"
person = pd.read_sql(sql_command,conn)
person.head()

Connected!
CPU times: user 14.5 s, sys: 1.05 s, total: 15.6 s
Wall time: 17.5 s


,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,death_datetime,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id
0,290150,8532,1978,None,None,None,NaT,0,0,None,None,None,290150,F,0,None,0,None,0
1,471383,8507,1954,None,None,None,NaT,0,0,None,None,None,471383,M,0,None,0,None,0
2,572322,8532,1999,None,None,None,NaT,0,0,None,None,None,572322,F,0,None,0,None,0
3,726625,8532,1963,None,None,None,NaT,0,0,None,None,None,726625,F,0,None,0,None,0
4,889391,8532,2008,None,None,None,NaT,0,0,None,None,None,889391,F,0,None,0,None,0


In [24]:
people_cohort = person[person.person_id.isin(non_recurrent_utis.person_id) ]
people_cohort = people_cohort[['person_id','year_of_birth']]

non_recurrent_utis_w_ages = non_recurrent_utis.merge(people_cohort, how = 'left', on = 'person_id')
non_recurrent_utis_w_ages['age'] = np.nan

In [25]:
%%time
i= 0 
for index,row in non_recurrent_utis_w_ages.iterrows():
    person_ = row['person_id']
    #temp = people_cohort[people_cohort['person_id']==person_]
    year_of_birth = row['year_of_birth']
    if year_of_birth >0:
        non_recurrent_utis_w_ages.loc[index,'age'] = row['condition_start_date'].year - year_of_birth
    i = i+1
    #if i%10000==0:
        #print(i) 

CPU times: user 1min 55s, sys: 224 ms, total: 1min 55s
Wall time: 1min 55s


In [26]:
# remove under 18
non_recurrent_utis_over_18 =non_recurrent_utis_w_ages[non_recurrent_utis_w_ages['age']>17]
print('# of events after removing under 18:')
print(len(non_recurrent_utis_over_18))
current_df = non_recurrent_utis_over_18

# of events after removing under 18:
135421


### Remove Patients with certain procedures

##### Find excluded procedures

In [ ]:
current_person_ids = current_df.person_id.unique()
print('# of current person IDs')
print(len(current_person_ids))

In [ ]:
%%time
## Load Procedures from cdm.procedure_occurence
current_person_str = ", ".join([f"'{x}'" for x in current_person_ids])
sql = """
    select
        p.procedure_concept_id as concept_id,
        p.procedure_date as date,
        person_id as person_id
    from
        {omop_schema}.procedure_occurrence p
    where 
        p.person_id in ({person_ids}) 
""".format(
    omop_schema=config.OMOP_CDM_SCHEMA,
    person_ids=current_person_str,
)
procedures = db.query(sql)
print(f"Procedures found: {len(procedures)}")

display(procedures.head(3))

In [ ]:
%%time

# get procedure names from cdm.concept
sql = """
    select
        c.concept_name as concept_name,
        c.concept_id as concept_id
    from
        {omop_schema}.concept c
    where 
        c.domain_id = 'Procedure' 
""".format(
    omop_schema=config.OMOP_CDM_SCHEMA,
    person_ids=current_person_ids,
)
procedure_names = db.query(sql)
procedure_names.head(3)

In [ ]:
%%time
procedure_name_hits = ["central venous catheter", "surgery", "surgical", "ventilation", "hemodialysis", "parenteral nutrition"]
exclude_name_hits = "than hemodialysis"
# Placement of a central venous cather

procedure_ids = []
for ind, (name, cid) in procedure_names[["concept_name", "concept_id"]].iterrows(): 
    name = name.lower()
    for pn in procedure_name_hits:
        if pn in name and exclude_name_hits not in name:
#             print(f"Procedure: {name}, code: {cid}")
            procedure_ids.append(cid)
            break
print(f"Found {len(procedure_ids)} procedure concept codes to exclude")
excluded_procedures = procedures[procedures.concept_id.isin(procedure_ids)]
print(f"Excluded procedure events: {len(excluded_procedures)}")


In [ ]:
### here

##### Find excluded conditions

In [ ]:
%%time
# get concept names from cdm.concept
sql = """
    select
        c.concept_name as concept_name,
        c.concept_id as concept_id,
        c.domain_id as domain_id
    from
        {omop_schema}.concept c
    where 
        c.domain_id = 'Condition' 
""".format(
    omop_schema=config.OMOP_CDM_SCHEMA,
)
diagnosis_names = db.query(sql)


In [ ]:
%%time
condition_ids = []
condition_name_hits = ["pregnancy", "pyelo","catheter"]
for ind, (name, cid) in diagnosis_names[["concept_name", "concept_id"]].iterrows(): 
    name = name.lower()
    for pn in condition_name_hits:
        if pn in name:
            condition_ids.append(cid)
            break
print(f"Found {len(condition_ids)} to exclude")

In [ ]:
%%time
excluded_conditions = condition_df[condition_df.concept_id.isin(condition_ids) & condition_df.person_id.isin(current_person_ids)]
print(len(excluded_conditions))
excluded_conditions = excluded_conditions[["concept_id", "condition_start_date", "person_id"]]
excluded_conditions = excluded_conditions.rename(columns={"condition_start_date":"date"})

##### Remove patients with the excluded conditions and procedures 

In [ ]:
%%time
excluded_table = pd.concat([excluded_conditions, excluded_procedures], ignore_index=True)
excluded_table

In [ ]:
%%time
#  Create in memory cache of events
person_events = dict()
c = 0
from datetime import datetime
start = datetime.now()

l = len(excluded_table)
for _, (_, date, pid) in excluded_table.iterrows():
    c += 1
    if c % 100000 == 0:
        print(f"Time elapsed: {(datetime.now() - start)}")
        print(f"Iter: {c} / {l}")
        
    if pid in person_events:
        person_events[pid].add(date)
    else:
        person_events[pid] = set([date])

In [ ]:
%%time
# Removes the event if a procedure happened in the last 180 days
#current_df.assign('No excluded procedure or condition')
#first_utis = utis[utis.no_previous_90_day_event].copy()
temp_df = current_df.assign(no_previous_180_excluded_event=True)
c = 0
from datetime import datetime
start = datetime.now()
l = len(temp_df)
for index, row in temp_df.iterrows():
    p = row.person_id
    d = row.condition_start_date
    if p in person_events.keys(): # added
        person_dates =  person_events[p]
        c += 1
        #if c % 10000 == 0:
            #print(f"Time elapsed: {(datetime.now() - start)}")
            #print(f"Iter: {c} ")
        for date in person_dates:
            if (d - date).days < 180 and (d - date).days > 0:
                temp_df.loc[index, "no_previous_180_excluded_event"] = False
                break
    
filtered_utis = temp_df[temp_df.no_previous_180_excluded_event].copy()

print(f"Final UTI list count: {len(filtered_utis)}")
filtered_utis.head(3)
current_df = filtered_utis

### Remove more conditions if they occurred ever

In [ ]:
%%time
condition_ids = []
condition_names_1 = []
condition_names_2 = []
condition_name_hits = [["neurogenic", "bladder"],["spina", "bifida"]]
for ind, (name, cid) in diagnosis_names[["concept_name", "concept_id"]].iterrows(): 
    name = name.lower()
    for lpn in condition_name_hits:
        pn = lpn[0]
        pn2 =lpn[1]
        #carcinoma OR cancer OR malignan) AND (bladder OR ureter OR kidney OR renal)
        if (('carcinoma' or 'cancer' or 'malignan') in name) and (('bladder' or 'ureter' or 'kidney' or 'renal')) in name:
            condition_ids.append(cid)
            condition_names_1.append(name)
            break
        elif pn in name and pn2 in name:
            condition_ids.append(cid)
            condition_names_2.append(name)
            break
print(f"Found {len(condition_ids)} to exclude")


excluded_conditions = condition_df[condition_df.concept_id.isin(condition_ids) & condition_df.person_id.isin(current_person_ids)]
print(len(excluded_conditions))
excluded_conditions = excluded_conditions[["concept_id", "condition_start_date", "person_id"]]
excluded_conditions = excluded_conditions.rename(columns={"condition_start_date":"date"})

excluded_table = excluded_conditions.copy()


In [ ]:
%%time
#  Create in memory cache of events
person_events = dict()
c = 0
from datetime import datetime
start = datetime.now()

l = len(excluded_table)
for _, (_, date, pid) in excluded_table.iterrows():
    c += 1
    if c % 100000 == 0:
        print(f"Time elapsed: {(datetime.now() - start)}")
        print(f"Iter: {c} / {l}")
        
    if pid in person_events:
        person_events[pid].add(date)
    else:
        person_events[pid] = set([date])

In [ ]:
%%time

temp_df = filtered_utis.assign(no_previous_excluded_event_ever=True)
c = 0
from datetime import datetime
start = datetime.now()
l = len(temp_df)
for index, row in temp_df.iterrows():
    p = row.person_id
    d = row.condition_start_date
    if p in person_events.keys(): # added
        person_dates =  person_events[p]
        c += 1
        #if c % 10000 == 0:
            #print(f"Time elapsed: {(datetime.now() - start)}")
            #print(f"Iter: {c} ")
        for date in person_dates:
            if (d - date).days > 0:
                temp_df.loc[index, "no_previous_excluded_event_ever"] = False
                break
    
filtered_utis = temp_df[temp_df.no_previous_excluded_event_ever].copy()

print(f"Final UTI list count: {len(filtered_utis)}")
filtered_utis.head(3)
current_df = filtered_utis

In [ ]:
filtered_utis = temp_df[temp_df.no_previous_excluded_event_ever].copy()

In [ ]:
filtered_utis['person_id'].value_counts()

In [ ]:
filtered_utis.to_sql("cohort_final_updated_treatments_ncj",con=db.engine, if_exists="replace", schema="cdm_6871_21")
db.execute("GRANT SELECT ON TABLE cdm_6871_21.cohort_final_updated_treatments_ncj TO cdm_6871_21;")


#### Get patients with 1 year of observation time

In [27]:
username='cdm_6871_21'
password='MLHC2021'
database_name = 'omop_v6'
conn_string = "dbname="+database_name + " host=/var/run/postgresql"
conn=psycopg2.connect(conn_string)

sql_command = "select * from cdm_6871_21.cohort_final_updated_treatments_ncj;"
cohort = pd.read_sql(sql_command,conn)
cohort.condition_occurrence_id.nunique()

104669

In [ ]:
# remove patients with insufficieny follow-up time

In [28]:
%%time
sql_cmd = 'select * from cdm_6871_21.visit_detail where person_id in (select person_id from cdm_6871_21.cohort_final_updated_treatments_ncj)'
visits = pd.read_sql(sql_cmd, conn)
visits.head()

CPU times: user 2min 22s, sys: 14.2 s, total: 2min 36s
Wall time: 4min 8s


,visit_detail_id,person_id,visit_detail_concept_id,visit_detail_start_date,visit_detail_start_datetime,visit_detail_end_date,visit_detail_end_datetime,visit_detail_type_concept_id,provider_id,care_site_id,discharge_to_concept_id,admitted_from_concept_id,admitted_from_source_value,visit_detail_source_value,visit_detail_source_concept_id,discharge_to_source_value,preceding_visit_detail_id,visit_detail_parent_id,visit_occurrence_id
0,19828375,1850532,581477,2013-05-01,2013-05-01,2013-05-01,2013-05-01,32021,524834,None,0,0,None,2294159961,0,~,None,None,38112738
1,406283058,4781602,581477,2020-06-06,2020-06-06,2020-06-06,2020-06-06,32021,4230642,None,0,0,None,4557351389,0,~,None,None,71271096
2,406283059,4781602,581477,2020-06-06,2020-06-06,2020-06-06,2020-06-06,32021,4230642,None,0,0,None,45573513819,0,~,None,None,71271096
3,406283060,4781602,581477,2020-06-06,2020-06-06,2020-06-06,2020-06-06,32021,4230642,None,0,0,None,4557351381,0,~,None,None,71271096
4,406283061,4781602,581477,2020-06-06,2020-06-06,2020-06-06,2020-06-06,32021,4230642,None,0,0,None,4557351384,0,~,None,None,71271096


In [29]:
first_visits = visits.groupby('person_id')[['visit_detail_start_date']].min()


In [30]:
first_visits.rename(columns={'visit_detail_start_date':'first_visit_date'}, inplace=True)
final = cohort.join(first_visits, on='person_id')

In [31]:
final['first_visit_date'] = pd.to_datetime(final['first_visit_date'])
final['condition_start_date_dt'] = pd.to_datetime(final['condition_start_date'])
final['followup_time'] = final['condition_start_date_dt'] - final['first_visit_date']

In [32]:
import datetime as dt

In [33]:
cohort = final[final.followup_time >= dt.timedelta(days=365)]

In [34]:
print('UTIs total with sufficient observation time')
print(len(cohort))

UTIs total with sufficient observation time
69513


In [ ]:
# remove multiple drrugs

In [ ]:
cohort['multi'].value_counts()

In [ ]:
final = cohort[cohort['multi']==False]

In [ ]:
print('UTIs total without multiple drigs')
print(len(final))

In [ ]:
# drop columns for formatting
final = final.drop(['first_visit_date','followup_time','condition_start_date_dt'],axis = 1)

In [ ]:
final = final.drop(['level_0'],axis=1)

In [ ]:
final.to_sql("cohort_manuscript_updated_treatments_ncj",con=db.engine, if_exists="replace", schema="cdm_6871_21")
db.execute("GRANT SELECT ON TABLE cdm_6871_21.cohort_manuscript_updated_treatments_ncj TO cdm_6871_21;")